In [1]:
# Import statements

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import random 
from copy import deepcopy

%matplotlib inline
import gym

In [2]:
# Register new environment
gym.envs.register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False}, # True ....
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

In [3]:
# Some basic parameters
inpDir = '../input'
outDir = '../output'

RANDOM_STATE = 42
STEPS = 200

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (9, 6),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

plt.rcParams.update(params)


SMALL_ENOUGH = 1e-9
GAMMA = 0.95

In [4]:
# Helper function to plot the grid

def fn_plot_heatmap(values, policy):
    fig, ax = plt.subplots()
    act = [action_mappings[action] for action in np.argmax(policy, axis=1)]
    labels = np.asarray(["{0}\n{1:.3f}".format(a, v)
                          for a, v in zip(act,values)]).reshape(4,4)
    values = values.reshape(4,4)
    sns.heatmap(values, annot=labels, fmt="", cmap='RdYlGn', ax=ax)
    plt.show()

In [5]:
# Load the frozen lake Environment
grid = gym.make('FrozenLakeNotSlippery-v0')
#grid = gym.make('FrozenLake-v0')
print('Environment Created.')

Environment Created.


In [6]:
grid.reset()
grid.render()


SFFF
FHFH
FFFH
HFFG


In [7]:
print(grid.observation_space)

Discrete(16)


In [8]:
print('Observation Space:' , grid.observation_space.n)

Observation Space: 16


In [9]:
print ('States:', grid.nS)

States: 16


In [10]:
# Randomly sample from our observation space
for i in range(10):
    print(grid.observation_space.sample())

14
15
15
0
13
14
9
9
9
5


In [11]:
# Action mappings - Map actions to numbers
action_mappings = {
    0: '\u2190' , # Left
    1: '\u2193', # Down
    2: '\u2192', # Right
    3: '\u2191', # Up
}

In [12]:
# Action space
print('Action Space:', grid.action_space)

A_n = grid.action_space.n
print('Action:', A_n)


Action Space: Discrete(4)
Action: 4


In [13]:
# Random Action sample

for i in range(10):
    a_n = grid.action_space.sample()
    print('Action: ', a_n, ',  Moving towards : ',action_mappings[a_n] )

Action:  2 ,  Moving towards :  →
Action:  3 ,  Moving towards :  ↑
Action:  0 ,  Moving towards :  ←
Action:  3 ,  Moving towards :  ↑
Action:  1 ,  Moving towards :  ↓
Action:  1 ,  Moving towards :  ↓
Action:  0 ,  Moving towards :  ←
Action:  2 ,  Moving towards :  →
Action:  2 ,  Moving towards :  →
Action:  2 ,  Moving towards :  →


In [14]:
# Go Down (action = 1)
grid.reset()
action = 1
grid.render()
observation, reward, done, prob = grid.step(action)
grid.render()
print(observation, reward, done, prob)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
4 0.0 False {'prob': 1.0}


In [15]:
# Stochastic
env = gym.make('FrozenLakeNotSlippery-v0')

In [16]:
# lets start with random policy
S_n = env.observation_space.n
A_n = env.action_space.n

policy = np.random.rand(S_n,A_n)
policy

array([[0.31501144, 0.51000558, 0.9067608 , 0.20275994],
       [0.07695813, 0.05616072, 0.36377082, 0.76970706],
       [0.99867101, 0.8902557 , 0.81140025, 0.37236358],
       [0.74287201, 0.02101717, 0.71843527, 0.85757703],
       [0.9487267 , 0.26430294, 0.74729791, 0.52327272],
       [0.17430708, 0.58759998, 0.79689051, 0.92856648],
       [0.21187249, 0.44247261, 0.14946588, 0.91876601],
       [0.03587508, 0.64934905, 0.87793958, 0.43862812],
       [0.47952204, 0.84081224, 0.47915377, 0.48456555],
       [0.16362087, 0.4077371 , 0.04883718, 0.14614614],
       [0.39292127, 0.03391366, 0.35552352, 0.74778607],
       [0.10655413, 0.41184159, 0.60405875, 0.28522234],
       [0.9259497 , 0.04458923, 0.43664197, 0.06012654],
       [0.22548075, 0.85782282, 0.95868314, 0.11531399],
       [0.99341278, 0.57714942, 0.93265043, 0.14411978],
       [0.33240808, 0.29035869, 0.64811989, 0.6306523 ]])

In [17]:
epsilon = 0.6
decay=0.9

In [18]:
def generate_episode(policy, env):
    
    # we initialize the list for storing states, actions, and rewards
    states, actions, rewards = [], [], []
    # Initialize the gym environment
    # observation means current state
    observation = env.reset()

    while True:
        # append the states to the states list
        states.append(observation)
        
        # now, we select an action using our sample_policy function and append the action to actions list
        action = random.choice([0,1,2,3])
        actions.append(action)
        
        # We perform the action in the environment according to our sample_policy, move to the next state 
        # and receive reward
        #print(observation)
        observation, reward, done, info = env.step(action)
        rewards.append(reward)
        
        # Break if the state is a terminal state
        if done:   #hole and gola are terminal states
             break
    #env.render()            
    return states, actions, rewards 

In [19]:
def first_visit_mc_prediction(policy, env, n_episodes):
    
    # First, we initialize the empty value table as a dictionary for storing the values of each state
    value_table = defaultdict(float)
    N = defaultdict(int)
    #states = [i for i in range(16)]
    #observation = random.choice(states)
    
    for _ in range(n_episodes):
        
        # Next, we generate the epsiode and store the states and rewards
        states, actions, rewards = generate_episode(policy, env)
        returns = 0
        
        # Then for each step, we store the rewards to a variable R and states to S, and we calculate
        # returns as a sum of rewards
        
        for t in range(len(states) - 1, -1, -1):
            R = rewards[t]
            S = states[t]
            A = actions[t]
            returns += R
            
            # Now to perform first visit MC, we check if the episode is visited for the first time, if yes,
            # we simply take the average of returns and assign the value of the state as an average of returns
            
            if S not in states[:t]: #not considering repeated states
                N[S] += 1
                value_table[S] += (returns - value_table[S]) / N[S]

    return value_table

In [21]:
value = first_visit_mc_prediction(policy, env,100000)

In [22]:
value

defaultdict(float,
            {6: 0.042279684945757234,
             4: 0.0160193707706617,
             2: 0.021593476013629972,
             1: 0.012042537349670562,
             0: 0.013940000000000109,
             3: 0.010144153764014875,
             8: 0.033813872745112834,
             9: 0.08596967838051008,
             13: 0.16978807294233608,
             10: 0.14603576960991324,
             14: 0.43332297171277573})

In [23]:
policy

array([[0.31501144, 0.51000558, 0.9067608 , 0.20275994],
       [0.07695813, 0.05616072, 0.36377082, 0.76970706],
       [0.99867101, 0.8902557 , 0.81140025, 0.37236358],
       [0.74287201, 0.02101717, 0.71843527, 0.85757703],
       [0.9487267 , 0.26430294, 0.74729791, 0.52327272],
       [0.17430708, 0.58759998, 0.79689051, 0.92856648],
       [0.21187249, 0.44247261, 0.14946588, 0.91876601],
       [0.03587508, 0.64934905, 0.87793958, 0.43862812],
       [0.47952204, 0.84081224, 0.47915377, 0.48456555],
       [0.16362087, 0.4077371 , 0.04883718, 0.14614614],
       [0.39292127, 0.03391366, 0.35552352, 0.74778607],
       [0.10655413, 0.41184159, 0.60405875, 0.28522234],
       [0.9259497 , 0.04458923, 0.43664197, 0.06012654],
       [0.22548075, 0.85782282, 0.95868314, 0.11531399],
       [0.99341278, 0.57714942, 0.93265043, 0.14411978],
       [0.33240808, 0.29035869, 0.64811989, 0.6306523 ]])

In [24]:
n_iteration = 1000
highscore = 0
data_row = []
for n_iter in range(n_iteration):

    points = 0 # keep track of the reward each episode
    data_dict = {}
    for i_episode in range(20): # run 20 episodes
        print ('_'*50)
        observation = env.reset()
        while True: # run until episode is done
            #grid.render()
            action = np.argmax(policy[observation])
            observation, reward, done, info = env.step(action)
            points += reward
            if done:
                env.render()
                #print (observation, reward, done, info)

                if points > highscore: # record high score
                    highscore = points
                break
    data_dict['run'] = n_iter
    data_dict['Points']=  points
    data_row.append(data_dict)

__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
F

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
____________________________________________

...

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
____________________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
____________________________________________

SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
____________________________________________

In [ ]:
prin(highscore)